<a href="https://colab.research.google.com/github/leehimcm/feature_selection/blob/main/BackwardSelection_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 작업 디렉토리 변경
import os
print(os.getcwd(), end=' -> ')
os.chdir('/content/drive/MyDrive/Colab Notebooks/25-1pattern_recognition')
print(os.getcwd())

/content -> /content/drive/MyDrive/Colab Notebooks/25-1pattern_recognition


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# CSV 파일 불러오기
df = pd.read_csv('train.csv')

# **데이터 전처리**

In [ ]:
# 많은 값들이 같은 값을 가지므로 삭제한다
df.drop(columns=['kw_min_min'], axis=1, inplace=True)
df.drop(columns=['kw_max_max'], axis=1, inplace=True)
df.drop(columns=['n_non_stop_words'], axis=1, inplace=True)

In [ ]:
# 'id'는 학습에 사용할 값이 아니므로 삭제한다
df.drop(columns=['id'], axis=1, inplace=True)

In [ ]:
# 결측치 처리
for col in df.columns:
    if df[col].dtype == 'object': # 범주형 → 최빈값
        df[col].fillna(df[col].mode()[0], inplace=True)
    else: # 숫자형 → 중간값
        df[col].fillna(df[col].median(), inplace=True)

<ipython-input-6-6a0114ce1937>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
<ipython-input-6-6a0114ce1937>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [ ]:
# is_weekend 변수 추가
df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday']).astype(int)

# 범주형 데이터 One-Hot Encoding
df = pd.get_dummies(df, columns=['data_channel', 'weekday'], drop_first=True)

In [ ]:
# 입력 변수(X), 라벨(y) 분리
X = df.drop(columns=['shares', 'y'])
y = np.log1p(df['shares'])  # log(1 + shares) -> shares=0 일 경우 에러 방지

# 정규화
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# train, val set을 나눈 뒤에 정규화 진행해보자.

# **Backward Selection**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

## 정규화

In [ ]:
# PCA 중요도 순서대로 feature가 담긴 list 저장
# ("preprocessing_v0_modified.ipynb" 마지막 셀의 출력 결과 복사, 붙여넣기)
important_features_ranked_index = [
 'data_channel_Lifestyle',
 'kw_avg_max',
 'global_rate_positive_words',
 'kw_min_avg',
 'data_channel_World',
 'min_positive_polarity',
 'LDA_00',
 'min_negative_polarity',
 'num_hrefs',
 'LDA_02',
 'global_rate_negative_words',
 'self_reference_min_shares',
 'num_keywords',
 'data_channel_Tech',
 'LDA_01',
 'kw_max_avg',
 'self_reference_max_shares',
 'LDA_04',
 'LDA_03',
 'num_self_hrefs',
 'kw_max_min',
 'num_videos',
 'abs_title_sentiment_polarity',
 'rate_positive_words',
 'num_imgs',
 'global_sentiment_polarity',
 'kw_min_max',
 'self_reference_avg_sharess',
 'data_channel_Entertainment',
 'max_negative_polarity',
 'average_token_length',
 'title_subjectivity',
 'data_channel_Social Media',
 'abs_title_subjectivity',
 'kw_avg_min',
 'n_non_stop_unique_tokens',
 'kw_avg_avg',
 'avg_negative_polarity',
 'rate_negative_words',
 'max_positive_polarity',
 'n_tokens_title',
 'n_unique_tokens',
 'avg_positive_polarity',
 'n_tokens_content',
 'title_sentiment_polarity',
 'weekday_Saturday',
 'weekday_Sunday',
 'is_weekend',
 'global_subjectivity',
 'weekday_Thursday',
 'weekday_Monday',
 'weekday_Tuesday',
 'weekday_Wednesday']

In [ ]:
# train, val set 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
# 정규화
scaler = StandardScaler()

# 학습 데이터에 scaler를 fit, transform
X_train_scaled_array = scaler.fit_transform(X_train)
# list를 이용해 feature을 선택할 수 있게끔 하기 위해서 pandas DataFrame으로 수정
X_train_scaled = pd.DataFrame(X_train_scaled_array, columns=important_features_ranked_index, index=X_train.index)

# 학습 데이터에 맞게 fit된 scaler로 검증 데이터를 transform
X_val_scaled_array = scaler.transform(X_val)
# list를 이용해 feature을 선택할 수 있게끔 하기 위해서 pandas DataFrame으로 수정
X_val_scaled = pd.DataFrame(X_val_scaled_array, columns=important_features_ranked_index, index=X_val.index)

## 함수 정의

#### **fixed_optional_split**: 고정 리스트와 선택 리스트 만들기

PCA 중요도 순위에서 몇 순위까지 고정하고 싶은지 바꾸고 싶을 때 사용한다. <br>
forward 함수를 호출하기 전에 사용해서, <br>
이 함수의 리턴값을 저장해서(fixed_features, optional_features) forward에 인수로 넣어주면 된다.

In [ ]:
# 고정 리스트와 선택 리스트 만드는 함수 정의
def fixed_optional_split(fixed_num):
    # 고정된 리스트: 고정 개수만큼, PCA 중요도 순서대로 저장
    fixed = important_features_ranked_index[:fixed_num]

    # 선택을 위한 리스트: 나머지를 저장
    optional = important_features_ranked_index[fixed_num:]

    return fixed, optional

In [ ]:
fixed_features = []
optional_features = []

In [ ]:
# 다음과 같이 사용
fixed_features, optional_features = fixed_optional_split(20)

In [ ]:
# 잘 들어갔는지 확인해보는 코드
fixed_features == important_features_ranked_index[:20]

True

#### **adjusted_r2**: Adjusted R^2를 계산하기

RidgePerf(성능 지표를 리턴하는 함수)의 내부에서 사용할 함수. <br>
- R2: R2 값 <br>
- N: number of examples (of validation set) (즉, 행의 수) <br>
- P: number of input variables(features) (of validation set) (즉, 열의 수)

In [ ]:
# R2값을 이용해 Adjusted R2를 계산하는 함수 adjusted_r2 정의
def adjusted_r2(R2, N, P):
    return 1 - (((1 - R2) * (N - 1)) / (N - P - 1))

#### **RidgePerf**: Ridge regression 모델을 학습시키고 성능 지표(Adjusted R2)를 리턴 받기

train과 validation set을 인수로 주면, Adjusted R2 값을 리턴하는 함수. <br>
- sklearn.linear_model 에서 import한 Ridge를 사용. <br>
- alpha는 1을 사용. (결정 과정은 아래 'Ridge 파라미터(alpha) 값 테스트'부분 참고)
- 나머지 인수들은 기본값.
- sklearn.metrics에서 import한 r2_score과, 위에서 정의한 adjusted_r2 함수를 사용.

alpha=1로 결정
(ForwardSelection_v0.ipynb 참고)

In [ ]:
# Ridge 모델을 학습시키고 Adjusted R2 성능 지표를 리턴하는 함수 RidgePerf 정의
def RidgePerf(X_tr, X_vl, y_tr, y_vl, alpha=1):
    model = Ridge(alpha = alpha)
    model.fit(X_tr, y_tr)
    y_predict = model.predict(X_vl)
    r2 = r2_score(y_vl, y_predict)
    r2_adj = adjusted_r2(r2, X_vl.shape[0], X_vl.shape[1])
    # 성능 지표로 Adjusted R2 사용
    metric = r2_adj
    return metric

In [ ]:
def backward_selection(X_trn, X_vld, y_trn, y_vld, current_features, remaining_features, alpha=1, max_no_improvement=3):
    # print(f"고정 개수: {len(current_features)}")
    # print(f"fixed_list: {current_features}")

    # 모든 feature로 학습한 성능 저장
    best_score = RidgePerf(X_trn[current_features], X_vld[current_features], y_trn, y_vld, alpha=alpha)
    print(f"Initial Adjusted R²: {best_score:.4f}")

    no_improvement_count = 0  # 성능이 개선되지 않은 횟수 저장
    improved = True

    while improved:
        improved = False
        scores_with_candidates = []

        # Backward Step: 제거
        for candidate in current_features:
            test_features = [f for f in current_features if f != candidate]
            if not test_features:
                continue
            score = RidgePerf(X_trn[test_features], X_vld[test_features], y_trn, y_vld, alpha=alpha)
            scores_with_candidates.append(('remove', candidate, score))

        # 가장 성능이 좋아진 경우 찾기
        scores_with_candidates.sort(key=lambda x: x[2], reverse=True)
        best_move, best_feature, best_move_score = scores_with_candidates[0]

        if best_move_score > best_score:
            improved = True
            best_score = best_move_score
            no_improvement_count = 0  # 성능이 개선되었으므로 리셋
            if best_move == 'add':
                current_features.append(best_feature)
                remaining_features.remove(best_feature)
                print(f"[Add] {best_feature} → Adjusted R²: {best_score:.4f}")
            elif best_move == 'remove':
                current_features.remove(best_feature)
                remaining_features.append(best_feature)
                print(f"[Remove] {best_feature} → Adjusted R²: {best_score:.4f}")
        else:
            no_improvement_count += 1
            if no_improvement_count >= max_no_improvement:
                improved = False
                print(f"No improvement for {max_no_improvement} iterations. Stopping...")
    print(f"# of features: {len(current_features)}")
    print(f"best Adjusted R²: {best_score:.4f}")
    return current_features, best_score


# Stepwise Selection 진행하기

In [ ]:
# Nlist: PCA 중요도가 가장 높은 1개, 중요도가 0.1보다 큰 6개, 중요도 시각화에서 큰 감소가 보이는 13, 21, 31개를 골라보았다.
# ('preprocessing_v0_modified.ipynb' 파일 참고)

N = len(important_features_ranked_index) # 모든 feature를 fixed_list에 포함시킨다
fixed_list, optional_list = fixed_optional_split(N)

selected_features, best_score = backward_selection(
    X_trn=X_train_scaled,
    X_vld=X_val_scaled,
    y_trn=y_train,
    y_vld=y_val,
    current_features=fixed_list,
    remaining_features=optional_list,
    alpha=1
)

# 결과 저장
result = (selected_features, best_score)
print(f"best_score: {best_score:.4f}")

# # 가장 좋은 성능을 보인 N값과 그 성능을 출력
# best_perf_subset = max(results_dict, key=lambda k: results_dict[k][1])
# best_perf = results_dict[best_perf_subset]
# print()
# print(f"best_perf_subset = {best_perf_subset}")
# print(f"best_perf[1] = {best_perf[1]:.4f}\n")


Initial Adjusted R²: 0.1002
[Remove] data_channel_Tech → Adjusted R²: 0.1022
[Remove] n_tokens_title → Adjusted R²: 0.1028
[Remove] kw_avg_min → Adjusted R²: 0.1033
[Remove] kw_avg_avg → Adjusted R²: 0.1039
[Remove] avg_positive_polarity → Adjusted R²: 0.1043
[Remove] self_reference_min_shares → Adjusted R²: 0.1047
[Remove] LDA_00 → Adjusted R²: 0.1050
[Remove] min_negative_polarity → Adjusted R²: 0.1053
[Remove] data_channel_Entertainment → Adjusted R²: 0.1056
[Remove] title_subjectivity → Adjusted R²: 0.1059
[Remove] abs_title_subjectivity → Adjusted R²: 0.1061
[Remove] weekday_Monday → Adjusted R²: 0.1064
[Remove] is_weekend → Adjusted R²: 0.1068
[Remove] num_videos → Adjusted R²: 0.1070
[Remove] weekday_Tuesday → Adjusted R²: 0.1073
[Remove] global_rate_negative_words → Adjusted R²: 0.1075
[Remove] global_rate_positive_words → Adjusted R²: 0.1077
[Remove] self_reference_avg_sharess → Adjusted R²: 0.1079
[Remove] num_self_hrefs → Adjusted R²: 0.1081
[Remove] global_subjectivity → Ad

In [ ]:
perf_list = [result]
df_perf = pd.DataFrame(perf_list, columns=['N', 'performance'])
print(df_perf)

                                                   N  performance
0  [kw_avg_max, data_channel_World, LDA_02, kw_ma...     0.109928


In [ ]:
# 가장 좋은 성능의 subset(list) 출력
result[0]

['kw_avg_max',
 'data_channel_World',
 'LDA_02',
 'kw_max_avg',
 'self_reference_max_shares',
 'LDA_04',
 'kw_max_min',
 'abs_title_sentiment_polarity',
 'rate_positive_words',
 'num_imgs',
 'global_sentiment_polarity',
 'max_negative_polarity',
 'average_token_length',
 'data_channel_Social Media',
 'avg_negative_polarity',
 'max_positive_polarity',
 'n_unique_tokens',
 'n_tokens_content',
 'title_sentiment_polarity',
 'weekday_Saturday',
 'weekday_Sunday']

# forward vs backward vs stepwise 결과 비교
    Adjusted R2
    - forward 0.109175
    - backward 0.109928
    - stepwise 0.109924

    

In [ ]:
'''
forward
['data_channel_Lifestyle',
 'self_reference_max_shares',
 'n_unique_tokens',
 'abs_title_sentiment_polarity',
 'avg_positive_polarity',
 'title_sentiment_polarity',
 'data_channel_World',
 'LDA_04',
 'weekday_Saturday',
 'kw_max_avg',
 'LDA_02',
 'global_sentiment_polarity',
 'global_rate_positive_words',
 'n_tokens_content',
 'kw_max_min',
 'num_hrefs',
 'num_videos',
 'n_tokens_title',
 'max_positive_polarity',
 'data_channel_Social Media',
 'weekday_Sunday']

backward
['kw_avg_max',
 'data_channel_World',
 'LDA_02',
 'kw_max_avg',
 'self_reference_max_shares',
 'LDA_04',
 'kw_max_min',
 'abs_title_sentiment_polarity',
 'rate_positive_words',
 'num_imgs',
 'global_sentiment_polarity',
 'max_negative_polarity',
 'average_token_length',
 'data_channel_Social Media',
 'avg_negative_polarity',
 'max_positive_polarity',
 'n_unique_tokens',
 'n_tokens_content',
 'title_sentiment_polarity',
 'weekday_Saturday',
 'weekday_Sunday']

 stepwise
 ['kw_avg_max',
 'data_channel_World',
 'LDA_02',
 'kw_max_avg',
 'self_reference_max_shares',
 'LDA_04',
 'kw_max_min',
 'abs_title_sentiment_polarity',
 'rate_positive_words',
 'num_imgs',
 'global_sentiment_polarity',
 'max_negative_polarity',
 'average_token_length',
 'n_unique_tokens',
 'title_sentiment_polarity',
 'weekday_Saturday',
 'n_tokens_content',
 'n_tokens_title',
 'max_positive_polarity',
 'data_channel_Social Media']


'''

In [ ]:
'''
backward, stepwise 공통 요소
[
    'kw_avg_max',
    'data_channel_World',
    'LDA_02',
    'kw_max_avg',
    'self_reference_max_shares',
    'LDA_04',
    'kw_max_min',
    'abs_title_sentiment_polarity',
    'rate_positive_words',
    'num_imgs',
    'global_sentiment_polarity',
    'max_negative_polarity',
    'average_token_length',
    'data_channel_Social Media',
    'max_positive_polarity',
    'n_unique_tokens',
    'n_tokens_content',
    'title_sentiment_polarity',
    'weekday_Saturday'
]


backward_only = [
    'avg_negative_polarity',
    'weekday_Sunday'
]


stepwise_only = [
    'n_tokens_title'
]


'''

향후 발전시킬 요소

    - 전처리 할 때 이상치 제거
    - AdjustedR2 이외에 다른 평가 지표(MSE, AIC, BIC 등)도 함께 비교